In [18]:
import os
import json
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
import google.auth

In [5]:
PROJECT_ID = "beam-udemy"
TOPIC_ID = "beam-example-input"
SUBSCRIPTION_ID = "beam-example-input-sub1"

In [6]:
options = PipelineOptions()

options.view_as(StandardOptions).streaming = True

_, options.view_as(GoogleCloudOptions).project = google.auth.default()

In [7]:
ib.options.recording_duration = '2m'

In [8]:
p = beam.Pipeline(InteractiveRunner(), options=options)

In [10]:
data = p | "read" >> beam.io.ReadFromPubSub(topic=f"projects/{PROJECT_ID}/topics/{TOPIC_ID}")

In [16]:
# ib.recordings.clear(p)

ib.show(data)

In [19]:
data_parsed = data | beam.Map(json.loads)

In [20]:
ib.show(data_parsed)

<IPython.core.display.Javascript object>

In [21]:
windowed = data_parsed | beam.WindowInto(beam.window.FixedWindows(10))

In [22]:
ib.show(windowed, include_window_info=True, visualize_data=True)

<IPython.core.display.Javascript object>

In [23]:
transactions = windowed | beam.Map(lambda e: e.get('transaction_amt'))

In [24]:
ib.show(transactions, include_window_info=True)

<IPython.core.display.Javascript object>

In [25]:
rates = transactions | beam.CombineGlobally(sum).without_defaults()

In [26]:
ib.show(rates, include_window_info=True)

<IPython.core.display.Javascript object>

In [28]:
ib.show_graph(p)